<div style="font-family:Arial Narrow" align="center">
<h1 style="font-size: 50pt">Job Market Analysis</h1> 
<h3 style="font-size: 30pt">An overview of Lombardia job market</h3>
<img src="./src/img1.png" />

</div>

<div style="font-family:Arial Narrow">
    <h1 style="font-size: 40pt">Data</h1>
    <p></p>
    <p style="font-size: 30pt">&nbsp;•&nbsp;Activated Contracts from Regione Lombardia</p>
    <p style="font-size: 30pt">&nbsp;•&nbsp;Ceased Contracts from Regione Lombardia</p>
    <p style="font-size: 30pt">&nbsp;•&nbsp;ATECO Code</p>
    <p style="font-size: 30pt">&nbsp;•&nbsp;GeoJSON Lombardia</p>
    <img width="200px" height="200px" src="./src/img2.png" align="right" />
</div>

In [ ]:
# Retriving all dataset from online sources if we haven't them

<div style="font-family:Arial Narrow">
    <h1 style="font-size: 40pt">Data Cleaning</h1>
    <p></p>
    <p style="font-size: 30pt">&nbsp;•&nbsp;Activated Contracts from Regione Lombardia</p>
    <p style="font-size: 30pt">&nbsp;•&nbsp;Ceased Contracts from Regione Lombardia</p>
    <p style="font-size: 30pt">&nbsp;•&nbsp;ATECO Code</p>
    <p style="font-size: 30pt">&nbsp;•&nbsp;GeoJSON Lombardia</p>
    <img width="200px" height="200px" src="./src/img3.png" align="right" />
</div>

In [ ]:
# Data Cleaning

<h1 style="font-family:Arial Narrow">Data Exploration</h1> 

In [ ]:
# Data Exploration

<h1 style="font-family:Arial Narrow">Prediction Models</h1> 

In [ ]:
# Prediction Models

<h1 style="font-family:Arial Narrow">Prediction Results</h1> 

In [ ]:
# Prediction Results

<h1 style="font-family:Arial Narrow">Conclusions</h1> 

<h1 style="font-family:Arial Narrow">Improvements</h1> 

<h1 style="font-family:Arial Narrow" align="center">Thanks for your attention!</h1>